In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

type X = ℝ
type Y = ℝ

viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

In [16]:
μ :: Linear ℝ Y (LocalLinear X Y)
μ = denseLinear $ \y -> denseLinear $ \δx -> y*δx

μ' :: Linear ℝ (LocalLinear X Y) Y
μ' = adjoint . denseLinear $ \λ -> denseLinear $ \δy -> δy*λ

deq :: DifferentialEqn X Y
deq = factoriseShade >>> snd >>> \(Shade y δy)
     -> let j = μ $ y   -- X-->Y
            δj = μ' `transformMetric` recipMetric δy -- Metric (X-->Y)
        in Shade' j δj
     -- δy :: Metric' Y ~ Metric Y' ~ Y'-->Y
     -- recipMetric δy :: Metric Y ~ Y-->Y'
     -- δj :: Metric (X-->Y) ~ (X-->Y)-->(X-->Y)' ~ (X-->Y)-->(Y-->X)
     --     ~ Metric' (Y-->X)
     -- (`transformMetric'`δy) :: (Y-->W) -> Metric' W
     -- (`transformMetric`recipMetric δy) :: (W-->Y) -> Metric W
     -- (μ' `transformMetric` recipMetric δy) :: Metric (X-->Y)
     -- μ :: Y --> (X-->Y)
     -- adjoint μ :: (X-->Y)' -> Y' ~ (Y-->X) -> Y'
     -- μ' :: (X-->Y) -> Y
     -- adjoint j :: Y'-->X'
     -- adjoint j' :: X'-->Y'
     -- adjoint j . recipMetric δy . j :: X-->X'
     -- j' . δy . adjoint j' :: X'-->X

In [17]:
skipN :: Int -> [a] -> [a]
skipN n xs = case splitAt n xs of
    ([], []) -> []
    ([x], []) -> [x]
    (xs₀, []) -> [head xs₀, last xs₀]
    (x:_, xs') -> x : skipN n xs'

In [18]:
tf :: PointsWeb X (Shade' Y)
tf = fromWebNodes euclideanMetric
        [(x, 1|±|[exp x*2-1.99]) | x<-[0, 0.02 .. 1.2] ]

tfs = iterateFilterDEqn_static deq tf
plotWindow $
      ( plot ((1,exp 1):±[(0.1,0),(0,0.1)] :: Shade ℝ²) ) -- Euler's number as reference for x=1
    : ( plot <$> reverse (take 10 $ skipN 10 tfs) )
-- Hask.toList . localFocusWeb $ head tfs
-- onlyLeaves tf'

GraphWindowSpecR2{lBound=-0.2193712943361397, rBound=1.5355990603529779, bBound=0.6065648111212638, tBound=3.7540463221511544, xResolution=640, yResolution=480}

![filtering the exponential function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential.png)

In [7]:
type YZ = ℝ²

μ₂ :: Linear ℝ YZ (LocalLinear X YZ)
μ₂ = denseLinear $ \(y,z) -> denseLinear $ \δx -> (z*δx, -y*δx)

deq₂ :: DifferentialEqn X YZ
deq₂ = factoriseShade >>> snd >>> \(Shade yz δyz)
     -> Shade' (μ₂ $ yz) $ recipMetric (projector' (μ₂ $ sqrt (fromDualWith δyz 1)))

No instance for (Floating interactive:Ghci79.YZ) arising from a use of ‘sqrt’
In the second argument of ‘($)’, namely ‘sqrt (fromDualWith δyz 1)’
In the first argument of ‘projector'’, namely ‘(interactive:Ghci79.μ₂ $ sqrt (fromDualWith δyz 1))’
In the first argument of ‘recipMetric’, namely ‘(projector' (interactive:Ghci79.μ₂ $ sqrt (fromDualWith δyz 1)))’

No instance for (Num (Double, Double)) arising from the literal ‘1’
In the second argument of ‘fromDualWith’, namely ‘1’
In the first argument of ‘sqrt’, namely ‘(fromDualWith δyz 1)’
In the second argument of ‘($)’, namely ‘sqrt (fromDualWith δyz 1)’

In [11]:
:t transformMetric'

transformMetric' :: forall v w.
(HasMetric' v, HasMetric' w, HasMetric' (DualSpace v), HasMetric' (DualSpace w), Scalar w ~ Scalar v, DualSpace (DualSpace v) ~ v, DualSpace (DualSpace w) ~ w) =>
Linear (Scalar w) v w -> HerMetric' v -> HerMetric' w